In [47]:
# import sys
# !{sys.executable} -m pip install fdsreader

In [48]:
# Import the required modules
import glob
import sys
import time
import os
from collections import defaultdict
import fdsreader as fds
import matplotlib.pyplot as plt
import numpy as np
from  scipy import interpolate
from scipy.integrate import solve_ivp

In [49]:

class windODE:
    def __init__(
        self,
        directory,
        fds_input_location,
        t_span,
    ):
        """

        :param directory: location of the fds output files
        :param fds_input_location: location of the fds input file
        :param t_span: time frame for ODE to run over

        :vars self.sim: fdsreader object
        :vars self.fds_input_location: fds_input_location
        :vars self.t_span: t_span
        :vars self.__directory: directory
        :vars self.__qFiles: list of all plot 3d output files
        :vars self.__timeList: list of all plot 3d time dumps
        :vars self.__voxalSize: resolution of each voxal
        :vars self.__maxVelocity: maximum velocity of any particle in the streamlines
        :vars self.startingpoints: list of all starting points to be used in the ODE

        """

        self.sim = fds.Simulation(directory)
        self.fds_input_location = fds_input_location
        self.t_span = t_span
        self.__directory = directory
        self.__qFiles = glob.glob(directory + "*.q")
        self.__timeList = np.array(self.sim.data_3d.times)
        self.__voxalSize = {}
        self.__maxVelocity = 0.0
        self.startingpoints = []
        self.__meshBounds = self.sim.meshes[0]
        self.__meshExtent = self.sim.meshes[0].extent

        self.getVoxalSize()



    def getVoxalSize(self):
        """
        Calculates voxal size
        :return:
        """


        self.__voxalSize["vx"] = (self.__meshExtent.x_end - self.__meshExtent.x_start) / (
            self.__meshBounds.dimension["x"] - 1
        )
        self.__voxalSize["vz"] = (self.__meshExtent.z_end - self.__meshExtent.z_start) / (
            self.__meshBounds.dimension["z"] - 1
        )
        self.__voxalSize["vy"] = (self.__meshExtent.y_end - self.__meshExtent.y_start) / (
            self.__meshBounds.dimension["y"] - 1
        )
        return self

    def getStartingPoints(self):

        """
        Creates a list of points  on the outer most voxals of OBSTS, one point per voxal
        :var X_Min_Value center point of minimum x voxal
        :var X_Max_Value center point of maximum x voxal
        :var Y_Min_Value center point of minimum y voxal
        :var Y_Max_Value center point of maximum y voxal

        :return:
        """
        X_Min_Value = (
                self.__meshExtent.x_start + self.__voxalSize["vx"] / 2.0
        )
        X_Max_Value = (
                self.__meshExtent.x_end - self.__voxalSize["vx"] / 2.0
        )
        Y_Min_Value = (
                self.__meshExtent.y_start + self.__voxalSize["vy"] / 2.0
        )
        Y_Max_Value = (
                self.__meshExtent.y_end - self.__voxalSize["vy"] / 2.0
        )
        with open(self.fds_input_location) as f:
            lines = f.readlines()

        lineCounter = 0
        while lineCounter < len(lines):
            current_line = lines[lineCounter]
            if current_line == "\n":
                lineCounter += 1
                continue
            while "/" not in lines[lineCounter]:
                lineCounter += 1
                current_line = current_line + lines[lineCounter]

            lineCounter += 1
            if "&OBST" not in current_line:
                continue
            mesh_line = current_line.replace("/", "").replace("\n", "")
            XB = [float(point) for point in mesh_line.split("XB=")[1].split(",")[:6]]

            if XB[0] <= X_Min_Value <= XB[1]:
                self.startingPointsRibbon(
                    [XB[0], XB[2], XB[5] + self.__voxalSize["vz"]],
                    [XB[0], XB[3], XB[5] + self.__voxalSize["vz"]],
                    int((XB[1] - XB[0]) / self.__voxalSize["vx"]),
                )

            if XB[0] <= X_Max_Value <= XB[1]:
                self.startingPointsRibbon(
                    [XB[1], XB[2], XB[5] + self.__voxalSize["vz"]],
                    [XB[1], XB[3], XB[5] + self.__voxalSize["vz"]],
                    int((XB[1] - XB[0]) / self.__voxalSize["vx"]),
                )
            if XB[2] <= Y_Min_Value <= XB[3]:
                self.startingPointsRibbon(
                    [XB[0], XB[2], XB[5] + self.__voxalSize["vz"]],
                    [XB[1], XB[2], XB[5] + self.__voxalSize["vz"]],
                    int((XB[3] - XB[2]) / self.__voxalSize["vy"]),
                )
            if XB[2] <= Y_Max_Value <= XB[3]:
                self.startingPointsRibbon(
                    [XB[0], XB[3], XB[5] + self.__voxalSize["vz"]],
                    [XB[1], XB[3], XB[5] + self.__voxalSize["vz"]],
                    int((XB[3] - XB[2]) / self.__voxalSize["vy"]),
                )

        return self

    def filterOutStreamsByLength(self):
        """
        This function removes all streamlines that total distance traveled is below a desired length.
        :return:
        """
        self.filteredTImeResults = {}

        self.distanceofWindStreams_index = defaultdict(lambda: [])
        allData = self.timeReasults
        for time in allData.keys():

            data = allData[time]
            numberofWindstreams = len(data)
            lengthofWindStreams = [len(x["y"][0]) for x in data]
            print(numberofWindstreams)
            print(lengthofWindStreams)
            for i in range(numberofWindstreams):
                distanceofWindStream = 0
                for j in range(1, len(data[i]["y"][0])):
                    point1 = np.array(
                        (
                            data[i]["y"][0][j - 1],
                            data[i]["y"][1][j - 1],
                            data[i]["y"][2][j - 1],
                        )
                    )
                    point2 = np.array(
                        (data[i]["y"][0][j], data[i]["y"][1][j], data[i]["y"][2][j])
                    )
                    p1_p2_distance = np.linalg.norm(point1 - point2)
                    distanceofWindStream += p1_p2_distance

                if distanceofWindStream > np.min(list(self.__voxalSize.values())) * 2.0:
                    self.distanceofWindStreams_index[time].append(i)


    def startingPointsRibbon(self, starting_pont, ending_point, number_of_points):
        x_ = np.linspace(starting_pont[0], ending_point[0], number_of_points)
        y_ = np.linspace(starting_pont[1], ending_point[1], number_of_points)
        z_ = np.linspace(starting_pont[2], ending_point[2], number_of_points)
        points = np.stack((x_.flatten(), y_.flatten(), z_.flatten()), axis=1)

        self.startingpoints.extend(points)
        return self

    def runODE(self, timedependent=True):
        self.timeReasults = {}
        for t_start in self.__timeList:
            if t_start > self.t_span[1] or t_start < self.t_span[0]:
                break

            closest_timeStep = min(self.__timeList, key=lambda x: abs(x - t_start))
            counter = np.where(self.__timeList == closest_timeStep)[0][0]
            all_results = []
            for startCounter in range(len(self.startingpoints)):

                y0 = self.startingpoints[startCounter]
                t_span = [
                    min(self.__timeList[counter:]),
                    max(self.__timeList[counter:]),
                ]
                # rtol=1E-4, atol=1E-6,
                result_solve_ivp = solve_ivp(
                    self.get_velocity, t_span, y0, args=[t_span[0], timedependent]
                )
                result_with_velocity = self.addVelocity(result_solve_ivp)
                current_result_max_vel = np.max(result_with_velocity["velocity"])
                if self.__maxVelocity < current_result_max_vel:
                    print(
                        f"new max Velocity {current_result_max_vel} changed from {self.__maxVelocity}"
                    )
                    self.__maxVelocity = current_result_max_vel
                all_results.append(result_with_velocity)
            self.timeReasults[t_start] = all_results
        return self

    def write2bin(self,desired_directory, file_name_prefix):
        fileName =os.path.join(desired_directory, file_name_prefix)
        allData = self.timeReasults
        maxVel = self.__maxVelocity
        print(f"Max {maxVel}")
        for time in allData.keys():
            data = allData[time]
            numberofWindstreams = len(data)
            lengthofWindStreams = [len(x["y"][0]) for x in data]
            print(numberofWindstreams)
            print(lengthofWindStreams)
            time_string = f"{time}".split(".")[1]
            with open(f"{fileName}_{int(time)}_{time_string}.binwind", "wb") as outfile:

                np.ndarray.tofile(np.array([maxVel], dtype=np.float32), outfile)
                np.ndarray.tofile(np.array([numberofWindstreams], dtype=int), outfile)
                np.ndarray.tofile(np.array(lengthofWindStreams, dtype=int), outfile)

                for i in range(numberofWindstreams):
                    currentStream = []
                    for j in range(len(data[i]["y"][0])):
                        currentStream.append(
                            [
                                data[i]["t"][j],
                                data[i]["velocity"][j],
                                data[i]["y"][0][j],
                                data[i]["y"][1][j],
                                data[i]["y"][2][j],
                            ]
                        )

                    np.ndarray.tofile(
                        np.array(currentStream, dtype=np.float32), outfile
                    )
                print(fileName, "saved")
        return self

    def get_velocity(self, t, x, t_0, timedependent):

        t = t if timedependent else t_0
        closest_timeStep = min(self.__timeList, key=lambda x: abs(x - t))
        counter = np.where(self.__timeList == closest_timeStep)[0][0]
        plt_3d_data = self.sim.data_3d[int(counter)]

        mesh = self.sim.meshes[0]
        # Select a quantity
        uvel_idx = plt_3d_data.get_quantity_index("U-VEL")
        vvel_idx = plt_3d_data.get_quantity_index("V-VEL")
        wvel_idx = plt_3d_data.get_quantity_index("W-VEL")
        index_values = self.get_index_values(x)
        u_vel_data = plt_3d_data[mesh].data[:, :, :, uvel_idx]
        u_velocity = u_vel_data[index_values[0], index_values[1], index_values[2]]
        v_vel_data = plt_3d_data[mesh].data[:, :, :, vvel_idx]
        v_velocity = v_vel_data[index_values[0], index_values[1], index_values[2]]
        w_vel_data = plt_3d_data[mesh].data[:, :, :, wvel_idx]
        w_velocity = w_vel_data[index_values[0], index_values[1], index_values[2]]

        return np.array([u_velocity, v_velocity, w_velocity])

    def get_index_values(self, x):
        x_index = (x[0] - self.__meshExtent.x_start) / self.__voxalSize["vx"]
        if 0 > x_index or x_index > self.__meshBounds.dimension["x"]:
            return np.array([0, 0, 0], dtype=int)
        y_index = (x[1] - self.__meshExtent.y_start) / self.__voxalSize["vy"]
        if 0 > y_index or y_index > self.__meshBounds.dimension["y"]:
            return np.array([0, 0, 0], dtype=int)
        z_index = (x[2] - self.__meshExtent.z_start) / self.__voxalSize["vz"]
        if 0 > z_index or z_index > self.__meshBounds.dimension["z"]:
            return np.array([0, 0, 0], dtype=int)
        return np.array([x_index, y_index, z_index], dtype=int)


    def addVelocity(self, oneDataSet):
        """

        :param oneDataSet:
        :return: same dataset with added velocity information
        """

        allSpeeds = [0.0] # all particles start at 0 velocity

        allTimes = oneDataSet["t"]
        allPositions = oneDataSet["y"]
        previousPosition = np.array(
            [allPositions[0][0], allPositions[1][0], allPositions[2][0]]
        )
        for i in range(1,len(allPositions[0])):
            currentPosition = np.array(
                [allPositions[0][i], allPositions[1][i], allPositions[2][i]]
            )

            deltaTime = allTimes[i] - allTimes[i - 1]
            squared_dist = np.sum((currentPosition - previousPosition) ** 2, axis=0)
            dist = np.sqrt(squared_dist)

            speed = dist / deltaTime
            allSpeeds.append(speed)
            previousPosition = currentPosition

        oneDataSet["velocity"] = np.array(allSpeeds)
        return oneDataSet

    def drawPlot(self):

        allData = self.timeReasults
        for time in self.distanceofWindStreams_index.keys():
            data = allData[time]

            fig = plt.figure(figsize=(8, 6))
            ax = fig.add_subplot(1, 1, 1, projection="3d")
            for i in self.distanceofWindStreams_index[time]:

                x = data[i]["y"][0][:]
                y = data[i]["y"][1][:]
                z = data[i]["y"][2][:]
                ax.plot(x, y, z)
            ax.set_title( f"Starting Time {time}")
            plt.show()

    def compairLines(self):
        allData = self.timeReasults
        for time in self.distanceofWindStreams_index.keys():

            plt.figure(figsize=(12, 8))
            data = allData[time]
            for i in self.distanceofWindStreams_index[time]:

                x = data[i]["y"][0][:]
                y = data[i]["y"][1][:]
                z = data[i]["y"][2][:]
                xy_est = np.polyfit(x, y,32)
                yz_est = np.polyfit(y,z,32)
                plt.subplot(2, 1, 1)
                plt.plot(x, y, 'o')
                # evaluate the values for a polynomial
                plt.plot(x, np.polyval(xy_est, x))
                plt.subplot(2, 1, 2)
                plt.plot( y,z, 'o')
                # evaluate the values for a polynomial
                plt.plot(y, np.polyval(yz_est, y))

            plt.tight_layout()
            plt.show()

    def showRE(self):

        allData = self.timeReasults
        for time in self.distanceofWindStreams_index.keys():
            data = allData[time]

            for i in self.distanceofWindStreams_index[time]:
                for j in range(len( data[i]["y"][0][:])):
                    x = data[i]["y"][0][j]
                    y = data[i]["y"][1][j]
                    z = data[i]["y"][2][j]
                    t = data[i]['t'][j]
                    temp = self.calculateRaynoldsNumber([x,y,z], t)
                    print(f"x {x} -  y {y} - z {z} -  RE {temp}")
            break

    def calculateRaynoldsNumber(self,x,t):
        closest_timeStep = min(self.__timeList, key=lambda x: abs(x - t))
        counter = np.where(self.__timeList == closest_timeStep)[0][0]
        plt_3d_data = self.sim.data_3d[int(counter)]

        mesh = self.sim.meshes[0]
        # Select a quantity
        rho_idx = plt_3d_data.get_quantity_index("rho")
        temp_idx = plt_3d_data.get_quantity_index("temp")
        uvel_idx = plt_3d_data.get_quantity_index("U-VEL")
        vvel_idx = plt_3d_data.get_quantity_index("V-VEL")
        wvel_idx = plt_3d_data.get_quantity_index("W-VEL")
        index_values = self.get_index_values(x)
        
        air_density_data = plt_3d_data[mesh].data[:, :, :, rho_idx]
        air_density = air_density_data[index_values[0], index_values[1], index_values[2]] 
                
        temp_data = plt_3d_data[mesh].data[:, :, :, temp_idx]
        temp_C = temp_data[index_values[0], index_values[1], index_values[2]] 
        
        u_vel_data = plt_3d_data[mesh].data[:, :, :, uvel_idx]
        u_velocity = u_vel_data[index_values[0], index_values[1], index_values[2]]
        
        v_vel_data = plt_3d_data[mesh].data[:, :, :, vvel_idx]
        v_velocity = v_vel_data[index_values[0], index_values[1], index_values[2]]
        
        w_vel_data = plt_3d_data[mesh].data[:, :, :, wvel_idx]
        w_velocity = w_vel_data[index_values[0], index_values[1], index_values[2]]
        
        flow_speed = np.sqrt(u_velocity*u_velocity +v_velocity*v_velocity + w_velocity*w_velocity)
        linear_demension = 20.0
        
        
        
        
        dynamic_viscosity_0 = 0.01827
        t_0 = 524.07
        sutherland_constant  = 120
        temperature_rankine = temp_C * (9/5) + 491.67 
        a = 0.555*t_0 + sutherland_constant
        b = 0.555*temperature_rankine + sutherland_constant
        
        mu = dynamic_viscosity_0 *(a/b)*np.power((temperature_rankine/t_0),1.5)
        print( air_density, flow_speed , linear_demension, mu)
        reynolds_number = air_density * flow_speed * linear_demension/ mu
        return reynolds_number, flow_speed
        
        
# The Reynolds number is defined as

# Re = uL/ν = ρuL/μ 

# where:

# ρ is the density of the fluid (SI units: kg/m3)
# u is the flow speed (m/s)
# L is a characteristic linear dimension (m) (see the below sections of this article for examples)
# μ is the dynamic viscosity of the fluid (Pa·s or N·s/m2 or kg/(m·s))
# ν is the kinematic viscosity of the fluid (m2/s).

# The Dynamic viscocity coefficient is defined as

# μ = μo*(a/b)*(T/To)3/2

# a = 0.555To + C
# b = 0.555T + C

# where

# μ  = viscosity in centipoise at input temperature T    
# μ0 = reference viscosity in centipoise at reference temperature To 0.01827
# T   = input temperature in degrees Rankine
# T0 = reference temperature in degrees Rankine 524.07
# C  = Sutherland's constant  = 120


In [50]:


fds_loc = "/home/trent/fds3/fds/trails.fds"
dir = "/home/trent/fds3/"
fds_loc = "/home/trent/Trunk/Trunk/Trunk.fds"
dir = "/home/trent/Trunk/TempCheck"

# fds_loc = "E:\Trunk\Trunk\Trunk\Trunk.fds"
# dir = "E:\Trunk\Trunk\\"

t_span = [0, 2]
start_time = time.perf_counter()
app = windODE(dir, fds_loc, t_span)
# app.getStartingPoints()
app.startingPointsRibbon([19,1,	3.5],[ 19,	19,	3.5],40)
app.runODE(timedependent=True)
app.filterOutStreamsByLength()
# app.write2bin("data","temp")
print(f"Total Time {time.perf_counter()-start_time:0.4f}")
# app.drawPlot()




new max Velocity 8.51079660979873 changed from 0.0
new max Velocity 8.584847088147447 changed from 8.51079660979873
new max Velocity 9.01027890128611 changed from 8.584847088147447
new max Velocity 9.081312515105523 changed from 9.01027890128611
40
[24, 21, 25, 20, 31, 30, 26, 54, 44, 34, 28, 17, 15, 16, 16, 21, 18, 19, 33, 35, 14, 17, 13, 12, 11, 11, 10, 11, 12, 32, 13, 10, 10, 8, 8, 8, 7, 7, 7, 6]
40
[29, 26, 26, 22, 32, 29, 28, 33, 42, 26, 28, 29, 23, 19, 17, 23, 19, 23, 23, 33, 17, 12, 15, 14, 12, 12, 12, 11, 13, 17, 13, 10, 9, 10, 8, 8, 7, 7, 7, 7]
40
[42, 30, 34, 27, 26, 29, 27, 28, 69, 30, 30, 24, 24, 19, 19, 19, 34, 31, 31, 57, 26, 13, 15, 14, 12, 12, 12, 12, 13, 34, 12, 14, 11, 9, 8, 8, 7, 7, 7, 6]
40
[35, 33, 27, 26, 28, 38, 39, 30, 35, 25, 26, 24, 19, 24, 23, 22, 19, 21, 31, 36, 21, 19, 15, 12, 14, 14, 13, 15, 13, 27, 24, 10, 8, 8, 7, 8, 9, 7, 7, 6]
Total Time 9.7779


In [51]:
app.showRE()

1.1438720226287842 6.06122582342984 20.0 0.018954882695889186
x 19.0 -  y 1.0 - z 3.5 -  RE (7315.546873587514, 6.06122582342984)
1.1438714265823364 5.926860855295677 20.0 0.018954890299705266
x 18.76849266372778 -  y 1.3567852520002788 - z 3.5097236554604896 -  RE (7153.369578517149, 5.926860855295677)
1.1438679695129395 6.0603056418186965 20.0 0.018954935922562758
x 18.445092852115472 -  y 1.8972705241276517 - z 3.5313347257208707 -  RE (7314.389811134442, 6.0603056418186965)
1.1438558101654053 7.372112566397199 20.0 0.018955091151052667
x 18.05918610158334 -  y 2.726760888025581 - z 3.596870715682384 -  RE (8897.486933792485, 7.372112566397199)
1.1438496112823486 8.406204274841981 20.0 0.018954839020275456
x 17.602757317138735 -  y 3.5308400156735322 - z 3.647226107402357 -  RE (10145.624008573915, 8.406204274841981)
1.1438554525375366 8.234102859141226 20.0 0.01895476817477593
x 17.12129408929238 -  y 4.207424160657917 - z 3.648445860860915 -  RE (9937.999099052502, 8.2341028591412

### The Reynolds number is defined as

    Re = uL/ν = ρuL/μ 

where:

    ρ is the density of the fluid (SI units: kg/m3)
    u is the flow speed (m/s)
    L is a characteristic linear dimension (m) (see the below sections of this article for examples)
    μ is the dynamic viscosity of the fluid (Pa·s or N·s/m2 or kg/(m·s))
    ν is the kinematic viscosity of the fluid (m2/s).

### The Dynamic viscocity coefficient is defined as

    μ = μo*(a/b)*(T/To)3/2

    a = 0.555To + C
    b = 0.555T + C

where

    μ  = viscosity in centipoise at input temperature T    
    μ0 = reference viscosity in centipoise at reference temperature To 0.01827
    T   = input temperature in degrees Rankine
    T0 = reference temperature in degrees Rankine 524.07
    C  = Sutherland's constant  = 120